# Has Many Movie Lab

### Introduction
In this lab we will continue to look at the "Has-Many" relationships in our data. The database we will be using during this lab contains information about a selection of movies and related entities such as actors, directors and writers. A movie entity will have relationships with actor, director, and writer entities. The actors, directors and writers will also have relationships with themselves (i.e. a director will have worked with many actors). In problems below, we will use our knowledge of these relationships to build SQL queries.

Let's begin by connecting to the database and reviewing the schema of the tables.

In [1]:
import sqlite3
conn = sqlite3.connect('movie_films_actors.db')
cursor = conn.cursor()

In [2]:
import pandas as pd
root_url = "https://raw.githubusercontent.com/jigsawlabs-student/curriculum-images/main/has-many-movies-lab/"
names = ['actors', 'directors', 'movies', 'writers', 'movie_actors', 'movie_directors', 'movie_writers']
loaded_dfs = [pd.read_csv(f'{root_url}{name}.csv') for name in names]

In [3]:
for index, name in enumerate(names):
    loaded_dfs[index].to_sql(f'{name}', conn, index = False)

In [4]:
cursor.execute('SELECT name from sqlite_master where type= "table"')
cursor.fetchall()

[('actors',),
 ('directors',),
 ('movies',),
 ('writers',),
 ('movie_actors',),
 ('movie_directors',),
 ('movie_writers',)]

In [5]:
cursor.execute('PRAGMA table_info(movies)')
cursor.fetchall()

[(0, 'id', 'INTEGER', 0, None, 0),
 (1, 'title', 'TEXT', 0, None, 0),
 (2, 'studio', 'TEXT', 0, None, 0),
 (3, 'runtime', 'REAL', 0, None, 0),
 (4, 'description', 'TEXT', 0, None, 0),
 (5, 'release_date', 'TEXT', 0, None, 0),
 (6, 'year', 'INTEGER', 0, None, 0)]

In [6]:
cursor.execute('PRAGMA table_info(actors)')
cursor.fetchall()

[(0, 'id', 'INTEGER', 0, None, 0), (1, 'name', 'TEXT', 0, None, 0)]

In [7]:
cursor.execute('PRAGMA table_info(directors)')
cursor.fetchall()

[(0, 'id', 'INTEGER', 0, None, 0), (1, 'name', 'TEXT', 0, None, 0)]

In [8]:
cursor.execute('PRAGMA table_info(writers)')
cursor.fetchall()

[(0, 'id', 'INTEGER', 0, None, 0), (1, 'name', 'TEXT', 0, None, 0)]

In [9]:
cursor.execute('PRAGMA table_info(movie_actors)')
cursor.fetchall()

[(0, 'id', 'INTEGER', 0, None, 0),
 (1, 'movie_id', 'INTEGER', 0, None, 0),
 (2, 'actor_id', 'INTEGER', 0, None, 0)]

In [10]:
cursor.execute('PRAGMA table_info(movie_directors)')
cursor.fetchall()

[(0, 'id', 'INTEGER', 0, None, 0),
 (1, 'movie_id', 'INTEGER', 0, None, 0),
 (2, 'director_id', 'INTEGER', 0, None, 0)]

In [11]:
cursor.execute('PRAGMA table_info(movie_writers)')
cursor.fetchall()

[(0, 'id', 'INTEGER', 0, None, 0),
 (1, 'movie_id', 'INTEGER', 0, None, 0),
 (2, 'writer_id', 'INTEGER', 0, None, 0)]

In [12]:
cursor.execute('PRAGMA table_info(movies)')
cursor.fetchall()

[(0, 'id', 'INTEGER', 0, None, 0),
 (1, 'title', 'TEXT', 0, None, 0),
 (2, 'studio', 'TEXT', 0, None, 0),
 (3, 'runtime', 'REAL', 0, None, 0),
 (4, 'description', 'TEXT', 0, None, 0),
 (5, 'release_date', 'TEXT', 0, None, 0),
 (6, 'year', 'INTEGER', 0, None, 0)]

Let's start off with some basic one table queries:

* What is the title, length, and id of the movie with the longest runtime?

In [15]:
query = '''
SELECT title, MAX(runtime), id 
FROM movies
'''

cursor.execute(query)
cursor.fetchall()

[('Never Sleep Again: The Elm Street Legacy', 480.0, 11415)]

* Using your answer from the previous question, how many actors were credited for the movie with the longest runtime? Hint: Use the COUNT function with the movie ID

In [19]:
query = '''
SELECT COUNT(actor_id)
FROM movie_actors
WHERE movie_id = 11415
'''

cursor.execute(query)
cursor.fetchall()


[(6,)]

* What was the shortest movie released in 2006?

In [21]:
query = '''
SELECT title, MIN(runtime), id 
FROM movies
WHERE year='2006'
'''

cursor.execute(query)
cursor.fetchall()

[('The Guardian', 15.0, 1632)]

### Has Many

* What are the names of the actors in Toy Story?

In [23]:
query = '''
SELECT actors.name 
FROM actors
JOIN movie_actors ON movie_actors.actor_id = actors.id
JOIN movies ON movie_actors.movie_id = movies.id
WHERE movies.title = 'Toy Story'
'''

cursor.execute(query)
cursor.fetchall()

[('Tom Hanks',),
 ('Tim Allen',),
 ('Jim Varney',),
 ('Wallace Shawn',),
 ('Don Rickles',),
 ('John Ratzenberger',)]

* What is the name of the director of Toy Story?

In [27]:
query = '''
SELECT directors.name 
FROM directors
JOIN movie_directors ON movie_directors.director_id = directors.id
JOIN movies ON movie_directors.movie_id = movies.id
WHERE movies.title = 'Toy Story'
'''

cursor.execute(query)
cursor.fetchall()

[('John Lasseter',)]

* What are the names of the writers of Toy Story?

In [28]:
query = '''
SELECT writers.name 
FROM writers
JOIN movie_writers ON movie_writers.writer_id = writers.id
JOIN movies ON movie_writers.movie_id = movies.id
WHERE movies.title = 'Toy Story'
'''

cursor.execute(query)
cursor.fetchall()

[('Joss Whedon',), ('Joel Cohen',), ('Andrew Stanton',), ('Alec Sokolow',)]

* What is the name and actor id of the actor with the most credits in the database?

In [31]:
query = '''
SELECT actors.name, actors.id, COUNT(movie_actors.actor_id) as credits
FROM actors
JOIN movie_actors ON movie_actors.actor_id = actors.id
GROUP BY movie_actors.actor_id
ORDER BY credits DESC
LIMIT 1
'''

cursor.execute(query)
cursor.fetchall()

[('Robert De Niro', 429, 78)]

* What are the titles of the movies the actor from the previous question has been in, after the year 2005?

In [36]:
query = '''
SELECT movies.title 
FROM movies
JOIN movie_actors ON movie_actors.movie_id = movies.id
WHERE year >= 2005 AND movie_actors.actor_id = 429
'''

cursor.execute(query)
cursor.fetchall()

[("New Year's Eve",),
 ('Mr. Warmth: The Don Rickles Project',),
 ('Hands of Stone',),
 ('Last Vegas',),
 ('I Knew It Was You: Rediscovering John Cazale',),
 ('Stardust',),
 ('Killer Elite',),
 ("Everybody's Fine",),
 ('Stone',),
 ('Machete',),
 ('Red Lights',),
 ('Righteous Kill',),
 ('The Good Shepherd',),
 ('The Bag Man',),
 ('Being Flynn',),
 ('Joy',),
 ('The Wizard of Lies',),
 ('Limitless',),
 ('Killing Season',),
 ('The Family',),
 ('Hide and Seek',),
 ('The Bridge of San Luis Rey',),
 ('Heist',),
 ('Great Expectations',),
 ('Little Fockers',),
 ('What Just Happened?',),
 ('The Comedian',),
 ('The Big Wedding',),
 ('Dirty Grandpa',),
 ('Grudge Match',)]

* What are the titles of movies with more than two directors?

### Has Many Through

* What is the name of the writer in the database that has been credited the most times during the year 2018?

* What is the name of the actor or actress in the database that has been credited the most between 2010 and 2015 (inclusive)?

* What are the names of all actors who performed in more than 3 movies in 2010?

* What studio has Steven Spielberg worked with the most?

* What years did Steven Spielberg direct 2 movies?

* How many movies has each of the actors from Toy Story been in? (movie ID is 3648)

* What are the names of other movies the director of Toy Story directed?

* What are the names of all the directors Tom Hanks has worked with? (Actor ID 189)

* What is the name of the director Tom Hanks has worked with the most?

* What are the names of all the writers Tom Hanks has worked with?

### Conclusion
The movie database we queried during this lab had a multitude of relationships between the tables. We saw how we could use JOIN to connect the tables, in order query information about entities in different tables. 